# Beehive Sensor Data Anylsis
- Consumes Silver data and testing out new data visualisations

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from hampel import hampel

Load Data

In [3]:
data = pd.read_csv(".streamlit/24-09-21_beehive_cleaned.csv")
print(data.dtypes)
data

created_at         object
entry_id            int64
weight            float64
temperature       float64
humidity           object
year                int64
month               int64
day                 int64
month_day          object
weight_cleaned    float64
dtype: object


C:\Users\timwy\AppData\Local\Temp\ipykernel_7180\3189062390.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(".streamlit/24-09-21_beehive_cleaned.csv")


,created_at,entry_id,weight,temperature,humidity,year,month,day,month_day,weight_cleaned
0,2020-10-14 15:03:37+00:00,1,49.63,78.0,13.1,2020,10,14,10-14,49.63
1,2020-10-14 15:13:41+00:00,2,49.77,95.0,20.3,2020,10,14,10-14,49.77
2,2020-10-14 15:23:45+00:00,3,49.79,95.0,23.1,2020,10,14,10-14,49.79
3,2020-10-14 15:33:49+00:00,4,49.78,86.0,23.4,2020,10,14,10-14,49.78
4,2020-10-14 15:43:53+00:00,5,49.77,79.0,23.5,2020,10,14,10-14,49.77
...,...,...,...,...,...,...,...,...,...,...
170768,2024-09-21 16:42:14+00:00,172760,42.73,75.0,28.90,2024,9,21,09-21,42.73
170769,2024-09-21 16:52:15+00:00,172761,42.73,75.0,28.90,2024,9,21,09-21,42.73
170770,2024-09-21 17:02:17+00:00,172762,42.73,75.0,28.80,2024,9,21,09-21,42.73
170771,2024-09-21 17:12:18+00:00,172763,42.73,75.0,28.80,2024,9,21,09-21,42.73
